## Importar librerias

In [4]:
# Importar librerias necesarias
import janitor
import matplotlib.pyplot as plt
import missingno
import numpy as np
import pandas as pd
import pyreadr
import seaborn as sns
import session_info
import upsetplot


## Configurar el aspecto general de las graficas del proyecto

In [5]:
%matplotlib inline

sns.set(rc={'figure.figsize':(10,10)})

sns.set_style("whitegrid")

### Operar con valores faltantes

#### Python